In [2]:
import numpy as np
import os
import pyreadr
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import torch
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# import tensorflow as tf

sns.set()


Sindy packages

In [3]:
import numpy as np
import pysindy as ps


Setup training and testing data

In [4]:
train_normal_path = './Dataset/TEP_FaultFree_Training.RData'
train_faulty_path = './Dataset/TEP_Faulty_Training.RData'

test_normal_path = './Dataset/TEP_FaultFree_Testing.RData'
test_faulty_path = './Dataset/TEP_Faulty_Testing.RData'


train_df = pyreadr.read_r(train_normal_path)['fault_free_training']    # (250000, 55)
test_df = pyreadr.read_r(test_faulty_path)['faulty_testing']

simulationRun = 1 | faultNumber = 0 (faultless)  dataset 설정
샘플이 총 500개있음. 500 샘플을 시간축으로 간주함.

In [5]:
feature_names = train_df.columns[3:]
train_normal_df = train_df[(train_df.faultNumber==0)&(train_df.simulationRun.isin(range(11)))][feature_names]   # (10000, 52)
X_np = train_normal_df.to_numpy()

dt = 1
n_states = len(train_normal_df.columns)
t = len(train_normal_df.index)
print(f'feature_names:{feature_names}')

feature_names:Index(['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6',
       'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12',
       'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18',
       'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24',
       'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30',
       'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36',
       'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1',
       'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9',
       'xmv_10', 'xmv_11'],
      dtype='object')


Scaler

In [6]:
scaler = preprocessing.MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler.fit_transform(X = train_normal_df))
train_df_scaled
X_np = train_df_scaled.to_numpy()


truncated SVD


In [7]:
SVD_RANK = 15
# get SVD rank
X1 = X_np[:-1, :]   # (499, 52)
X2 = X_np[1:, :]

U, s, Vh = np.linalg.svd(X1.T, full_matrices=False)



differentiation submodule

In [8]:
differentiation_method = ps.FiniteDifference(order=2)


Feature_library submodule

In [9]:
feature_library = ps.PolynomialLibrary(degree=3)

# Note: We could instead call ps.feature_library.PolynomialLibrary(degree=3)

Next we select which optimizer should be used.

In [10]:
optimizer = ps.STLSQ(threshold=0.2)

# Note: We could instead call ps.optimizers.STLSQ(threshold=0.2)

Finally, we bring these three components together in one `SINDy` object.

In [11]:
model = ps.SINDy(
    differentiation_method=differentiation_method,
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=feature_names,
)

Following the `scikit-learn` workflow, we first instantiate a `SINDy` class object with the desired properties, then fit it to the data in separate step.

In [12]:
model.fit(X_np, t=dt)

/home/sihoon/anaconda3/envs/HAVOK_test/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.2) and eliminated all coefficients
  warnings.warn(
/home/sihoon/anaconda3/envs/HAVOK_test/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.2) and eliminated all coefficients
  warnings.warn(
/home/sihoon/anaconda3/envs/HAVOK_test/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.2) and eliminated all coefficients
  warnings.warn(
/home/sihoon/anaconda3/envs/HAVOK_test/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.2) and eliminated all coefficients
  warnings.warn(
/home/sihoon/anaconda3/envs/HAVOK_test/lib/python3.10/site-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.2) and eliminated all coefficients
  warnings.warn(
/home

SINDy(differentiation_method=FiniteDifference(),
      feature_library=PolynomialLibrary(degree=3),
      feature_names=Index(['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6',
       'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12',
       'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18',
       'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24',
       'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30',
       'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36',
       'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1',
       'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9',
       'xmv_10', 'xmv_11'],
      dtype='object'),
      optimizer=STLSQ(threshold=0.2))

We can inspect the governing equations discovered by the model and check whether they seem reasonable with the `print` function.

In [13]:
model.print()

(xmeas_1)' = 0.000
(xmeas_2)' = 0.415 xmeas_9 xmeas_31 xmv_11 + -0.472 xmeas_26 xmeas_31 xmv_11
(xmeas_3)' = 0.312 xmeas_6 xmeas_10 xmeas_37 + -0.313 xmeas_6 xmeas_37 xmv_6
(xmeas_4)' = 0.000
(xmeas_5)' = 0.000
(xmeas_6)' = 0.000
(xmeas_7)' = 0.000
(xmeas_8)' = 0.000
(xmeas_9)' = 0.000
(xmeas_10)' = -0.666 xmeas_1 xmeas_40^2 + 0.658 xmeas_40^2 xmv_3
(xmeas_11)' = 0.000
(xmeas_12)' = 0.000
(xmeas_13)' = 0.000
(xmeas_14)' = 0.000
(xmeas_15)' = 0.000
(xmeas_16)' = 0.000
(xmeas_17)' = 0.000
(xmeas_18)' = 0.000
(xmeas_19)' = 0.000
(xmeas_20)' = 0.000
(xmeas_21)' = 0.000
(xmeas_22)' = 0.000
(xmeas_23)' = 2.224 xmeas_1 xmeas_6 xmv_4 + -2.148 xmeas_6 xmv_3 xmv_4
(xmeas_24)' = 0.000
(xmeas_25)' = 0.000
(xmeas_26)' = 0.480 xmeas_2^2 xmeas_21 + -0.436 xmeas_2 xmeas_21^2
(xmeas_27)' = 0.000
(xmeas_28)' = 0.000
(xmeas_29)' = 0.000
(xmeas_30)' = 0.477 xmeas_4 xmeas_19 xmeas_26 + -0.522 xmeas_4 xmeas_26 xmv_9
(xmeas_31)' = 0.000
(xmeas_32)' = 0.000
(xmeas_33)' = 0.000
(xmeas_34)' = 0.000
(xmeas_35)' 